# 🧠 Week 1: LangGraph Foundations

## Blueprint to Your First Agent

Welcome to Week 1 of the Multi-Agent Mastery course! In this notebook, you'll learn the fundamental building blocks of LangGraph and build your first working graph.

---

## 🚀 Getting Started

### Prerequisites
- Python 3.8 or higher
- Jupyter Notebook installed

### Setup Instructions

**Option 1: Using requirements.txt (Recommended)**
```bash
pip install -r requirements.txt
```

**Option 2: Manual Installation**
```bash
pip install langgraph langchain-core langchain-openai ipython jupyter
```

**Note:** This notebook does NOT require an OpenAI API key. All examples use simple functions that don't call external APIs.

---

### 📚 What You'll Learn

1. **State** — The shared data structure that flows through your graph
2. **Nodes** — Functions that process and transform state
3. **Edges** — Control flow that connects nodes together
4. **StateGraph** — The orchestrator that ties everything together

### 🎯 By the End of This Notebook

- ✅ Understand the core LangGraph architecture
- ✅ Build a simple echo agent from scratch
- ✅ Create a Customer Support Router with conditional routing
- ✅ Visualize your graph execution
- ✅ Be ready to tackle more complex multi-agent systems

---

### 🔑 Key Concepts

| Concept | Analogy | Purpose |
|---------|---------|---------|
| **State** | Shared notebook | Data that all nodes can read/write |
| **Nodes** | Workers | Functions that do the actual work |
| **Edges** | Arrows | Define which node runs next |
| **StateGraph** | Manager | Orchestrates the entire flow |

---

### 📖 How to Use This Notebook

1. **Run cells in order** - Each cell builds on the previous one
2. **Read the markdown cells** - They contain important explanations
3. **Experiment** - Try modifying the code to see what happens
4. **Ask questions** - If something doesn't work, check the error messages

**Ready? Let's start by installing dependencies in the next cell!**


In [ ]:
# Install dependencies (run once)
# If you have requirements.txt in the same directory, you can use:
# %pip install -r requirements.txt

# Otherwise, install individually:
%pip install langgraph langchain-core langchain-openai ipython

# Verify installation
import sys
print(f"Python version: {sys.version}")
print("✅ Dependencies installed successfully!")


---

## 🛠️ Setup & Imports

Let's import the essential components. Here's what each import does:

| Import | Purpose |
|--------|---------|
| `operator` | Python's built-in module — we use `operator.add` for list concatenation |
| `Annotated` | Type hint that lets us attach metadata (like reducers) to our state fields |
| `TypedDict` | Creates a dictionary with typed keys — the foundation of our State |
| `StateGraph` | The main class for building graphs in LangGraph |
| `START, END` | Special constants representing the entry and exit points of our graph |
| `AIMessage, HumanMessage` | LangChain message types for chat-based interactions |


In [ ]:
# Imports
import operator
from typing import Annotated
from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langchain_core.messages import AIMessage, HumanMessage


---

## 📓 Step 1: Define STATE

**State is the shared memory of your graph.** Think of it as a notebook that every node can read from and write to.

### Key Concepts:

- **TypedDict**: Defines the shape/schema of your state
- **Annotated**: Attaches a "reducer" function that determines *how* updates are merged
- **`operator.add`**: A reducer that *appends* new items to a list (instead of replacing)

### Why Reducers Matter

Without a reducer, returning `{"messages": [new_msg]}` would **replace** the entire list.
With `operator.add`, it **appends** the new message to existing messages.

```
Before: {"messages": [msg1, msg2]}
Node returns: {"messages": [msg3]}

Without reducer → {"messages": [msg3]}        # REPLACED! 😱
With operator.add → {"messages": [msg1, msg2, msg3]}  # APPENDED! ✅
```


In [ ]:
# 1. Define STATE - the shared data structure
class State(TypedDict):
    messages: Annotated[list, operator.add]  # Append-only!


---

## ⚡ Step 2: Define NODES

**Nodes are functions that do the actual work.** They:
1. Receive the current state as input
2. Perform some operation (call an LLM, execute a tool, process data)
3. Return updates to the state

### Node Function Signature

```python
def my_node(state: State) -> dict:
    # Read from state
    messages = state["messages"]
    
    # Do some work...
    
    # Return ONLY the fields you want to update
    return {"messages": [new_message]}
```

### Important Rules:
- Nodes **only return what changes** — you don't need to return the entire state
- The reducer (if defined) handles merging the update with existing state
- Node names must be unique within a graph


In [ ]:
# 2. Define NODES - functions that process state
def echo_node(state):
    return {"messages": [AIMessage("Echo!")]}


---

## 🔀 Step 3: Build the GRAPH with EDGES

**Edges define the control flow** — which node runs after which.

### Building a Graph:

1. **Create the StateGraph** with your State class
2. **Add nodes** — register your functions with unique names
3. **Add edges** — connect nodes to define execution order
4. **Compile** — creates an executable application

### Edge Types:

| Type | Method | Use Case |
|------|--------|----------|
| **Normal Edge** | `add_edge(A, B)` | A always goes to B |
| **Conditional Edge** | `add_conditional_edges(A, fn)` | fn decides where A goes |
| **Entry Point** | `add_edge(START, A)` | A is the first node |
| **Exit Point** | `add_edge(A, END)` | A is the last node |

### Special Constants:
- `START` — Where execution begins
- `END` — Where execution terminates


In [ ]:
# 3. Build the graph with EDGES
graph = StateGraph(State)

# Add nodes
graph.add_node("echo", echo_node)

# Add edges (control flow)
graph.add_edge(START, "echo")  # START → echo → END
graph.add_edge("echo", END)

# Compile the graph
app = graph.compile()


---

## 🚀 Step 4: Run the Graph!

Now we can invoke our compiled graph with an initial state.

### Execution Flow:

```
invoke({"messages": [HumanMessage("hi")]})
        │
        ▼
    ┌───────┐
    │ START │
    └───┬───┘
        │
        ▼
    ┌───────┐
    │ echo  │ → Returns {"messages": [AIMessage("Echo!")]}
    └───┬───┘
        │
        ▼
    ┌───────┐
    │  END  │
    └───────┘
        │
        ▼
Result: {"messages": [HumanMessage("hi"), AIMessage("Echo!")]}
```

The `invoke()` method runs the graph synchronously and returns the final state.


In [ ]:
# 4. Run the graph!
result = app.invoke({"messages": [HumanMessage("hi")]})
result


---

## 👁️ Visualize Your Graph

LangGraph can render your graph as a diagram using Mermaid. This is incredibly useful for:
- Understanding the flow of your application
- Debugging complex multi-agent systems
- Documentation and presentations


In [ ]:
# Bonus: Visualize the graph
from IPython.display import Image, display
display(Image(app.get_graph().draw_mermaid_png()))


---

## 📬 Inspect the Messages

Let's look at what's in our result. The state now contains both the original human message AND the AI response.


In [ ]:
# See the messages
for msg in result["messages"]:
    print(f"{msg.type}: {msg.content}")


---

# 🎯 Worked Example: Customer Support Router

Now let's build something more practical — a **Customer Support Router** that demonstrates conditional edges!

## The Architecture

```
                    ┌─────────────┐
                    │    START    │
                    └──────┬──────┘
                           │
                           ▼
                    ┌─────────────┐
                    │  Classifier │  ← Determines ticket type
                    └──────┬──────┘
                           │
              ┌────────────┼────────────┐
              │            │            │
              ▼            ▼            ▼
        ┌──────────┐ ┌──────────┐ ┌──────────┐
        │ Technical│ │  Billing │ │  General │
        │  Support │ │  Support │ │  Support │
        └────┬─────┘ └────┬─────┘ └────┬─────┘
              │            │            │
              └────────────┼────────────┘
                           │
                           ▼
                    ┌─────────────┐
                    │     END     │
                    └─────────────┘
```

## What We'll Learn:
1. **Conditional Edges** — Route to different nodes based on state
2. **Classifier Pattern** — LLM decides the routing
3. **Multiple Nodes** — Specialized handlers for different cases


### Step 1: Extended State for Routing

Our state needs additional fields to track the ticket category and response.


In [ ]:
from typing import Literal

# Extended state for the support router
class SupportState(TypedDict):
    messages: Annotated[list, operator.add]  # Chat history (append-only)
    ticket_category: str                      # Category determined by classifier
    response: str                             # Final response to user


### Step 2: Define the Nodes

We need 4 nodes:
1. **Classifier** — Analyzes the message and determines category
2. **Technical Support** — Handles technical issues
3. **Billing Support** — Handles billing questions
4. **General Support** — Handles everything else


In [ ]:
# Node 1: Classifier - determines the ticket category
def classifier_node(state: SupportState) -> dict:
    """Analyze the message and categorize it."""
    last_message = state["messages"][-1].content.lower()
    
    # Simple keyword-based classification (in production, use an LLM!)
    if any(word in last_message for word in ["error", "bug", "crash", "not working", "broken"]):
        category = "technical"
    elif any(word in last_message for word in ["bill", "charge", "payment", "invoice", "refund"]):
        category = "billing"
    else:
        category = "general"
    
    print(f"📋 Classified as: {category}")
    return {"ticket_category": category}


# Node 2: Technical Support
def technical_support_node(state: SupportState) -> dict:
    """Handle technical issues."""
    print("🔧 Technical Support handling the request...")
    response = "🔧 Technical Support: I'll help you troubleshoot this issue. Have you tried turning it off and on again?"
    return {
        "response": response,
        "messages": [AIMessage(response)]
    }


# Node 3: Billing Support
def billing_support_node(state: SupportState) -> dict:
    """Handle billing questions."""
    print("💳 Billing Support handling the request...")
    response = "💳 Billing Support: I can help with your billing inquiry. Let me pull up your account details."
    return {
        "response": response,
        "messages": [AIMessage(response)]
    }


# Node 4: General Support
def general_support_node(state: SupportState) -> dict:
    """Handle general inquiries."""
    print("📞 General Support handling the request...")
    response = "📞 General Support: Thank you for reaching out! How can I assist you today?"
    return {
        "response": response,
        "messages": [AIMessage(response)]
    }


### Step 3: Define the Routing Function

The routing function reads the state and returns the name of the next node to execute.


In [ ]:
# Routing function - decides which support node to call
def route_to_support(state: SupportState) -> Literal["technical", "billing", "general"]:
    """Route to the appropriate support team based on ticket category."""
    return state["ticket_category"]


### Step 4: Build the Graph with Conditional Edges

Now we connect everything using `add_conditional_edges()` which allows dynamic routing.


In [ ]:
# Build the Support Router Graph
support_graph = StateGraph(SupportState)

# Add all nodes
support_graph.add_node("classifier", classifier_node)
support_graph.add_node("technical", technical_support_node)
support_graph.add_node("billing", billing_support_node)
support_graph.add_node("general", general_support_node)

# Add edges
support_graph.add_edge(START, "classifier")  # Start with classification

# Add CONDITIONAL edges from classifier to support teams
support_graph.add_conditional_edges(
    "classifier",           # Source node
    route_to_support,       # Routing function
    {                       # Mapping: return value → target node
        "technical": "technical",
        "billing": "billing",
        "general": "general"
    }
)

# All support nodes go to END
support_graph.add_edge("technical", END)
support_graph.add_edge("billing", END)
support_graph.add_edge("general", END)

# Compile!
support_app = support_graph.compile()
print("✅ Support Router compiled successfully!")


### Visualize the Support Router


In [ ]:
# Visualize the support router graph
display(Image(support_app.get_graph().draw_mermaid_png()))


### Test the Router with Different Queries

Let's test all three routing paths!


In [ ]:
# Test Case 1: Technical Issue
print("=" * 50)
print("TEST 1: Technical Issue")
print("=" * 50)
result1 = support_app.invoke({
    "messages": [HumanMessage("My app keeps crashing with an error!")],
    "ticket_category": "",
    "response": ""
})
print(f"Response: {result1['response']}\n")


In [ ]:
# Test Case 2: Billing Question
print("=" * 50)
print("TEST 2: Billing Question")
print("=" * 50)
result2 = support_app.invoke({
    "messages": [HumanMessage("I need a refund for my last payment")],
    "ticket_category": "",
    "response": ""
})
print(f"Response: {result2['response']}\n")


In [ ]:
# Test Case 3: General Inquiry
print("=" * 50)
print("TEST 3: General Inquiry")
print("=" * 50)
result3 = support_app.invoke({
    "messages": [HumanMessage("What are your business hours?")],
    "ticket_category": "",
    "response": ""
})
print(f"Response: {result3['response']}")


---

# 📝 Summary

Congratulations! You've learned the core building blocks of LangGraph:

## Key Takeaways

| Concept | What You Learned |
|---------|------------------|
| **State** | Shared data structure with reducers for controlled updates |
| **Nodes** | Functions that read state, do work, and return updates |
| **Edges** | Control flow — normal edges for fixed paths, conditional edges for dynamic routing |
| **StateGraph** | The builder class that ties everything together |

## Patterns Covered

1. ✅ **Simple Linear Graph**: START → Node → END
2. ✅ **Conditional Routing**: Classifier → Dynamic branching based on state

## What's Next?

- **Homework**: Build a Calculator Agent with the ReAct pattern
- **Week 2**: Single-agent mastery with persistence and memory
- **Week 3**: Multi-agent orchestration with subgraphs

---

### 🔗 Resources

- [LangGraph Documentation](https://langchain-ai.github.io/langgraph/)
- [LangSmith](https://smith.langchain.com) — Debug & trace your graphs
- [LangGraph Academy](https://academy.langchain.com/courses/intro-to-langgraph)


# 🎉 End of Week 1 Notebook - Happy Learning!
